## Вставьте содержимое вашего локального файла ключей `~/.ssh/id_rsa.pub` (c:\Users\username\\.ssh\id_rsa.pub для Windows), а также токен аутентификации ngrok в следующую ячейку

- Если у вас нет  такого файла, используйте команду `ssh-keygen` для генерации.

- ngrok authtoken можно скопировать отсюда: https://dashboard.ngrok.com/auth после бесплатной регистрации.




In [0]:
# Вставьте содержимое локального id_rsa.pub сюда
authorized_key = '''SSH-ключи вставлять сюда'''

# Вставьте сюда authtoken из  https://dashboard.ngrok.com/auth
ngrok_auth = 'NGROK-TOKEN вставлять сюда'

In [3]:
#@title Installing ssh and creating sshd_config here. Just run this cell!
!apt-get install ssh
sshd_config = '''
#	$OpenBSD: sshd_config,v 1.101 2017/03/14 07:19:07 djm Exp $

# This is the sshd server system-wide configuration file.  See
# sshd_config(5) for more information.

# This sshd was compiled with PATH=/usr/bin:/bin:/usr/sbin:/sbin

# The strategy used for options in the default sshd_config shipped with
# OpenSSH is to specify options with their default value where
# possible, but leave them commented.  Uncommented options override the
# default value.

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

#HostKey /etc/ssh/ssh_host_rsa_key
#HostKey /etc/ssh/ssh_host_ecdsa_key
#HostKey /etc/ssh/ssh_host_ed25519_key

# Ciphers and keying
#RekeyLimit default none

# Logging
#SyslogFacility AUTH
#LogLevel INFO

# Authentication:

#LoginGraceTime 2m
#PermitRootLogin prohibit-password
#StrictModes yes
#MaxAuthTries 6
#MaxSessions 10

#PubkeyAuthentication yes

# Expect .ssh/authorized_keys2 to be disregarded by default in future.
#AuthorizedKeysFile	.ssh/authorized_keys .ssh/authorized_keys2

#AuthorizedPrincipalsFile none

#AuthorizedKeysCommand none
#AuthorizedKeysCommandUser nobody

# For this to work you will also need host keys in /etc/ssh/ssh_known_hosts
#HostbasedAuthentication no
# Change to yes if you don't trust ~/.ssh/known_hosts for
# HostbasedAuthentication
#IgnoreUserKnownHosts no
# Don't read the user's ~/.rhosts and ~/.shosts files
#IgnoreRhosts yes

# To disable tunneled clear text passwords, change to no here!
#PasswordAuthentication yes
#PermitEmptyPasswords no

# Change to yes to enable challenge-response passwords (beware issues with
# some PAM modules and threads)
ChallengeResponseAuthentication no

# Kerberos options
#KerberosAuthentication no
#KerberosOrLocalPasswd yes
#KerberosTicketCleanup yes
#KerberosGetAFSToken no

# GSSAPI options
#GSSAPIAuthentication no
#GSSAPICleanupCredentials yes
#GSSAPIStrictAcceptorCheck yes
#GSSAPIKeyExchange no

# Set this to 'yes' to enable PAM authentication, account processing,
# and session processing. If this is enabled, PAM authentication will
# be allowed through the ChallengeResponseAuthentication and
# PasswordAuthentication.  Depending on your PAM configuration,
# PAM authentication via ChallengeResponseAuthentication may bypass
# the setting of "PermitRootLogin without-password".
# If you just want the PAM account and session checks to run without
# PAM authentication, then enable this but set PasswordAuthentication
# and ChallengeResponseAuthentication to 'no'.
UsePAM yes

#AllowAgentForwarding yes
AllowTcpForwarding yes
#GatewayPorts no
X11Forwarding yes
#X11DisplayOffset 10
#X11UseLocalhost yes
#PermitTTY yes
PrintMotd no
#PrintLastLog yes
#TCPKeepAlive yes
#UseLogin no
#PermitUserEnvironment no
#Compression delayed
#ClientAliveInterval 0
#ClientAliveCountMax 3
#UseDNS no
#PidFile /var/run/sshd.pid
#MaxStartups 10:30:100
#PermitTunnel no
#ChrootDirectory none
#VersionAddendum none

# no default banner path
#Banner none

# Allow client to pass locale environment variables
AcceptEnv LANG LC_*

# override default of no subsystems
Subsystem	sftp	/usr/lib/openssh/sftp-server

# Example of overriding settings on a per-user basis
#Match User anoncvs
#	X11Forwarding no
#	AllowTcpForwarding no
#	PermitTTY no
#	ForceCommand cvs server'''

ssh_path = '/root/.ssh'
ngrok_path = '/content/ngrok-ssh'

ssh_yml='''
tunnels:
  ssh:
    proto: tcp
    addr: 22'''   

http5000_yml = '''
tunnels:
  http5000:
    proto: http
    addr: 5000    
    inspect: false
    bind_tls: true'''

run_ngrok_sh = '''#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --config ''' + ngrok_path + '''/ssh.yml --log=stdout --config ''' + ngrok_path + '''/http5000.yml "$@"
'''

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  ncurses-term openssh-server openssh-sftp-server python3-certifi
  python3-chardet python3-idna python3-pkg-resources python3-requests
  python3-six python3-urllib3 ssh-import-id
Suggested packages:
  molly-guard monkeysphere rssh ssh-askpass ufw python3-setuptools
  python3-cryptography python3-openssl python3-socks
The following NEW packages will be installed:
  ncurses-term openssh-server openssh-sftp-server python3-certifi
  python3-chardet python3-idna python3-pkg-resources python3-requests
  python3-six python3-urllib3 ssh ssh-import-id
0 upgraded, 12 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,154 kB of archives.
After this operation, 7,710 kB of additional disk space will be use

In [4]:
#@title Installing and running ngrok. Just run this cell.

import os

if not os.path.exists(ssh_path): os.makedirs(ssh_path)
if not os.path.exists(ngrok_path): os.makedirs(ngrok_path)

with open(os.path.join(ssh_path, 'authorized_keys'), "w") as authorized_keys_file:
    authorized_keys_file.write(authorized_key+'\n')
with open('/etc/ssh/sshd_config', "w") as sshd_config_file:
    sshd_config_file.write(sshd_config)

!service ssh restart

#!mkdir -p /drive/ngrok-ssh
%cd {ngrok_path}
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp "$ngrok_path/ngrok" /ngrok
!chmod +x /ngrok
!/ngrok authtoken {ngrok_auth}

with open(os.path.join(ngrok_path, 'ssh.yml'), "w") as ssh_yml_file:
    ssh_yml_file.write(ssh_yml)

with open(os.path.join(ngrok_path, 'http5000.yml'), "w") as http5000_yml_file:
    http5000_yml_file.write(http5000_yml)

with open(os.path.join(ngrok_path, 'run_ngrok.sh'), "w") as run_ngrok_sh_file:
    run_ngrok_sh_file.write(run_ngrok_sh)

ngrok_str ='''bash ''' + ngrok_path + '''/run_ngrok.sh ssh http5000 & '''

get_ipython().system_raw(ngrok_str)

 * Restarting OpenBSD Secure Shell server sshd
   ...done.
/content/ngrok-ssh
--2020-02-27 08:48:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.1.17.102, 52.70.189.149, 34.237.57.234, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.1.17.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  63.0MB/s    in 0.2s    

2020-02-27 08:48:37 (63.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [0]:
# https://colab.research.google.com/github/shawwn/colab-tricks/blob/master/ngrok-tricks.ipynb
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

In [6]:
for name, tunnel in get_ngrok_tunnels():
  local_url = get_ngrok_url(name, local=True)
  public_url = get_ngrok_url(name, local=False)
  print('{:12s} {} <-> {}'.format(name, public_url, local_url))

http5000     https://1ab0f68a.ngrok.io <-> http://localhost:5000
ssh          tcp://0.tcp.ngrok.io:15673 <-> localhost:22


In [6]:
public_url, public_port = urllib.parse.urlparse(get_ngrok_url('ssh')).netloc.split(':')
print('To SSH into this colab instance, run the following command on your local machine:')
print('')

print('  ssh -p {} root@{}'.format(public_port, public_url))  

To SSH into this colab instance, run the following command on your local machine:

  ssh -p 14877 root@0.tcp.ngrok.io


In [9]:
!pip install flask_babel;
%cd /content/
!git clone https://github.com/tuuka/cv-demo
%cd cv-demo
!ls -la

/content
fatal: destination path 'cv-demo' already exists and is not an empty directory.
/content/cv-demo
total 40
drwxr-xr-x 5 root root 4096 Feb 26 15:45 .
drwxr-xr-x 1 root root 4096 Feb 26 15:44 ..
drwxr-xr-x 9 root root 4096 Feb 26 15:45 app
-rw-r--r-- 1 root root  254 Feb 26 15:44 config.py
-rw-r--r-- 1 root root  574 Feb 26 15:45 config.pyc
-rw-r--r-- 1 root root  116 Feb 26 15:44 cv-demo.py
drwxr-xr-x 8 root root 4096 Feb 26 15:44 .git
-rw-r--r-- 1 root root  107 Feb 26 15:44 .gitignore
drwxr-xr-x 2 root root 4096 Feb 26 15:45 __pycache__
-rw-r--r-- 1 root root   87 Feb 26 15:44 README.md


In [10]:
# To run app uncomment next line and run this cell. 
#get_ipython().system_raw('python3 cv-demo.py')


KeyboardInterrupt



In [21]:
# To terminate the tunnel uncomment next line and run this cell. 
#!kill $(ps aux | grep './ngrok' | awk '{print $2}')

^C


### Чтобы настроить удаленный интерпретатор в Pro версии PyCharm, раскомментируйте и запустите следующую ячейку, чтобы подключить ваш GoogleDrive для синхронизации проекта. В настройках удаленного интерпретатора PyCharm (помимо адреса и порта для ssh- подключения) необходимо будет указать mapping локальной папки проекта к папке /drive, которая, собственно, маунтится в ячейке ниже.

In [0]:
# mounting google drive

'''
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p /drive
path = "/content/gdrive/My Drive/Colab Notebooks"
!mount --bind "$path" /drive
'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
